In [1]:

import numpy as np 
import pandas as pd 
import json

import re
import os
from bs4 import BeautifulSoup
import requests
import csv
import itertools
from tqdm import tqdm


In [7]:
def get_links(tag,page_limit):
    hrefs=[]
    for page in range(1,page_limit):
        req=requests.get(url='https://stackoverflow.com/questions/tagged/{}?tab=newest&page={}&pagesize=15'.format(tag,page))
        soup=BeautifulSoup(req.text,"html.parser")
        for i in soup.find_all("a",class_="s-link",href=True):
            hrefs.append(i['href'])
            
    filtered_links = [link for link in hrefs if link != "javascript:void(0)" and not link.startswith("https")]
    prefix='https://stackoverflow.com'
    final_links=[]
    for h in filtered_links:
        if 'https' not in h:
            m=prefix+h
            final_links.append(m)
            
    return final_links

In [24]:
def questions_and_anwers(links):
    questions_json=[]
    margin=1e-7#epsilon for 0 votes
    for link in tqdm(links,total=len(links)):
        question_dict = {
                'question': None,
                'answers': None,
                'votes':None
            }


        response = requests.get(url=link)
        soup = BeautifulSoup(response.text,"html.parser")


        reply_answers=[]
        answer_votes=[]
        marg_votes=[]
        question_elements = soup.find_all('div',{'class':'question js-question'})
        for question in question_elements:#getting question
            post_body = question.find(class_="s-prose js-post-body")
            asked_question=re.sub(r'<[^>]+>', '', str(post_body))


            
            question_dict['question']=re.sub(r'\s+', ' ', asked_question).strip()

        answer_elements = soup.find_all('div',{'id':'answers'})
        for answer in answer_elements:
            post_body = answer.find_all(class_="s-prose js-post-body")
            vote_elem=answer.find_all(class_='js-vote-count')
           
            for elem in post_body:#getting answers
                reply_answer=re.sub(r'<[^>]+>', '', str(elem))

                local_answers=[]
                local_answers.append( re.sub(r'\s+', ' ', reply_answer).strip())
                reply_answers.append(local_answers)
            for elem in vote_elem:#getting votes
                votes=elem.get_text()

                answer_votes.append( re.sub(r'\s+', ' ', votes).strip())
                int_votes=[eval(i) for i in answer_votes]
                marg_votes=[margin+i for i in int_votes]
        question_dict['answers']=reply_answers
        question_dict['votes']=marg_votes
        questions_json.append(question_dict)
    return questions_json

In [13]:
tags1=['nvidia','nvidia-jetson','nvidia-jetson-nano','nvidia-docker','nvidia-digits','nvidia-deepstream','nvidia-smi','nvidia-titan','nvidia-flex','nvidia-hpc-compilers','nvidia-isaac','nvidia-jetpack-sdk','nvidia-sass','nvidia-modulus']
topic_pages1=[75,9,7,7,3,3,2,2,2,2,2,2,2,2]

In [14]:
tags2=['pycuda','theano-cuda','cuda-gdb','jcuda','cuda-streams','cudafy.net','cuda-context','cuda-driver','cuda-arrays','managed-cuda','cuda-graphs','cuda.net','cuda-events','cudamat','cuda-uva','scikit-cuda','cudatoolkit','cuda-profiling','cuda-jit-cache','cuda-wmma','triton','tritonserver']
topic_pages2=[10,4,3,3,3,3,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2]

In [17]:
tags=tags1+tags2
topic_pages=topic_pages1+topic_pages2

In [ ]:
cuda_page_lim=287

In [26]:
for i in range(0,len(tags)):#saving tag wise jsons
    links=get_links(tags[i],topic_pages[i])
    q_and_a=questions_and_anwers(links)
    with open('{}.json'.format(tags[i]), 'w') as json_file:
       json.dump(q_and_a, json_file, indent=4)

100%|██████████| 38/38 [00:26<00:00,  1.44it/s]
